Extract and Process - Flights Test Data 2020

In [1]:
import re
import pandas as pd
import psycopg2

In [2]:
my_vars = """
- Y **fl_date**: Flight Date (yyyy-mm-dd)
- Y **mkt_unique_carrier**: Unique Marketing Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **branded_code_share**: Reporting Carrier Operated or Branded Code Share Partners
- Y **mkt_carrier**: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
- **mkt_carrier_fl_num**: Flight Number
- Y **op_unique_carrier**: Unique Scheduled Operating Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users,for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- **tail_num**: Tail Number
- **op_carrier_fl_num**: Flight Number
- Y **origin_airport_id**: Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- Y **origin**: Origin Airport
- Y **origin_city_name**: Origin Airport, City Name
- Y **dest_airport_id**: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- Y **dest**: Destination Airport
- Y **dest_city_name**: Destination Airport, City Name
- Y **crs_dep_time**: CRS Departure Time (local time: hhmm)
- Y **crs_arr_time**: CRS Arrival Time (local time: hhmm)
- **dup**: Duplicate flag marked Y if the flight is swapped based on Form-3A data
- Y **crs_elapsed_time**: CRS Elapsed Time of Flight, in Minutes
- **flights**: Number of Flights
- Y **distance**: Distance between airports (miles)
"""

In [3]:
feat_name = '- Y.+\*'
feat_list = re.findall(feat_name, my_vars)

feat_list_clean=[]
for feat in feat_list:
    feat = re.sub(r"[- Y **]", "", feat)
    feat_list_clean.append(feat)

In [4]:
param_dic = {
    'host': '<enter host here>',
    'database': '<enter database>',
    'user': '<user>',
    'port': '<port>',
    'password': '<password>'
}

In [5]:
def connect(param_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**param_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgres_to_df(conn, select_query, column_names):
    """
    Transforms a SELECT query into a pandas dataframe
    """
    cursor = con.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        cursor.close()
        return 1
    
    # list of tuples
    res_tuples = cursor.fetchall()
    cursor.close()

    # return to dataframe
    df = pd.DataFrame(res_tuples, columns=column_names)
    return df

In [6]:
con = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
query = """
SELECT fl_date, mkt_unique_carrier, mkt_carrier, 
op_unique_carrier, origin_airport_id, origin, 
origin_city_name, dest_airport_id, dest, 
dest_city_name, crs_dep_time, crs_arr_time, 
crs_elapsed_time, distance
FROM flights_test
WHERE fl_date >= '2020-01-01 00:00:00'::timestamp
AND fl_date <= '2020-01-07 00:00:00'::timestamp;
"""

col_names = ['fl_date', 'mkt_unique_carrier', 'mkt_carrier', 
'op_unique_carrier', 'origin_airport_id', 'origin', 'origin_city_name', 
'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time', 
'crs_elapsed_time', 'distance']

In [8]:
df = postgres_to_df(con, query, col_names)

### Processing data

In [9]:
# convert dates to datettime
df['fl_date'] = pd.to_datetime(df['fl_date'])
df['fl_day'] = df['fl_date'].dt.strftime("%A")

In [10]:
#split origin city and state
df[['origin_city','origin_state']]=df['origin_city_name'].str.split(',', expand=True)
df[['dest_city','dest_state']]=df['dest_city_name'].str.split(',', expand=True)

In [11]:
# convert crs_times to datettime 
# pad front with 0
df['crs_dep_hour'] = df['crs_dep_time'].astype(str).str.zfill(4)
df['crs_arr_hour'] = df['crs_arr_time'].astype(str).str.zfill(4)
# replace 2400 (not recognized by dataframe) to 0000 (midnight)
df['crs_dep_hour'].replace('2400', '0000', inplace=True)
df['crs_arr_hour'].replace('2400', '0000', inplace=True)
# convert to datetimes and extract hour oly
df['crs_dep_hour'] = pd.to_datetime(df['crs_dep_hour'], format="%H%M").dt.round('H').dt.hour
df['crs_arr_hour'] = pd.to_datetime(df['crs_arr_hour'], format="%H%M").dt.round('H').dt.hour

In [12]:
# convert hour to time of day categorical variables 
df['dep_tod'] = pd.cut(df['crs_dep_hour'],
    bins=[0,6,12,18,24],
    labels=['overnight','morning','afternoon','evening'],
    right=False,
    include_lowest=True)

df['arr_tod'] = pd.cut(df['crs_arr_hour'],
    bins=[0,6,12,18,24],
    labels=['overnight','morning','afternoon','evening'],
    right=False,
    include_lowest=True)

In [13]:
# getting week_num 
df['week_num'] = df['fl_date'].dt.isocalendar().week

In [14]:
df.columns

Index(['fl_date', 'mkt_unique_carrier', 'mkt_carrier', 'op_unique_carrier',
       'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id',
       'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance', 'fl_day', 'origin_city', 'origin_state',
       'dest_city', 'dest_state', 'crs_dep_hour', 'crs_arr_hour', 'dep_tod',
       'arr_tod', 'week_num'],
      dtype='object')

In [15]:
# extract only relevant features
df = df[['mkt_unique_carrier','op_unique_carrier','origin',
'origin_city','dest','dest_city',
'crs_elapsed_time','crs_dep_hour','dep_tod',
'crs_arr_hour','arr_tod','distance',
'fl_date','fl_day','week_num']]

### Merge with cleaned passenger data 

In [16]:
# note that path may be different
df_pass = pd.read_csv('data/processed/toJoin_passenger.csv', index_col=[0])

In [17]:
df['identifier']= df['op_unique_carrier'] + "-" + df['origin'] + "-" + df['dest']

In [18]:
df_fl_pass = pd.merge(df, df_pass, left_on='identifier', 
    right_on='identifier1', how='left').drop('identifier1', axis=1)

In [19]:
# drop any missing values
df_fl_pass = df_fl_pass.dropna()
# drop redundant columns
joined_cols_toDrop = ['distance_y', 'dest_y', 'origin_y', 'unique_carrier']
df_fl_pass = df_fl_pass.drop(joined_cols_toDrop, axis=1)
# rename original columns
df_fl_pass = df_fl_pass.rename(columns={'origin_x':'origin','dest_x':'dest','distance_x':'distance'})

In [20]:
df_fl_pass.columns

Index(['mkt_unique_carrier', 'op_unique_carrier', 'origin', 'origin_city',
       'dest', 'dest_city', 'crs_elapsed_time', 'crs_dep_hour', 'dep_tod',
       'crs_arr_hour', 'arr_tod', 'distance', 'fl_date', 'fl_day', 'week_num',
       'identifier', 'departures_performed', 'payload', 'passengers',
       'freight', 'air_time', 'aircraft_type', 'distance_group'],
      dtype='object')

In [21]:
# extract relevant columns 
df_fl_pass = df_fl_pass[['identifier','mkt_unique_carrier', 'op_unique_carrier',
'origin','origin_city','dest',
'dest_city','crs_elapsed_time', 'crs_dep_hour', 
'dep_tod','crs_arr_hour', 'arr_tod',
'distance','fl_date', 'fl_day', 'week_num', 
'departures_performed', 'payload', 'passengers','freight', 
'air_time', 'distance_group']]

In [22]:
df_fl_pass.head()

,identifier,mkt_unique_carrier,op_unique_carrier,origin,origin_city,dest,dest_city,crs_elapsed_time,crs_dep_hour,dep_tod,...,distance,fl_date,fl_day,week_num,departures_performed,payload,passengers,freight,air_time,distance_group
2,WN-ONT-SJC,WN,WN,ONT,Ontario,SJC,San Jose,70,20,evening,...,333,2020-01-01,Wednesday,1,13.0,418600.0,1112.0,5.0,720.0,1.0
3,WN-ONT-SJC,WN,WN,ONT,Ontario,SJC,San Jose,75,14,afternoon,...,333,2020-01-01,Wednesday,1,13.0,418600.0,1112.0,5.0,720.0,1.0
4,WN-ONT-SJC,WN,WN,ONT,Ontario,SJC,San Jose,80,9,morning,...,333,2020-01-01,Wednesday,1,13.0,418600.0,1112.0,5.0,720.0,1.0
5,WN-ONT-SJC,WN,WN,ONT,Ontario,SJC,San Jose,75,6,morning,...,333,2020-01-01,Wednesday,1,13.0,418600.0,1112.0,5.0,720.0,1.0
6,WN-ONT-SJC,WN,WN,ONT,Ontario,SJC,San Jose,80,16,afternoon,...,333,2020-01-01,Wednesday,1,13.0,418600.0,1112.0,5.0,720.0,1.0


### Merge with cleaned Fuel Data

In [23]:
df_fuel = pd.read_csv('data/processed/toJoin_fuel.csv', index_col=[0])

In [24]:
df_fpf = pd.merge(df_fl_pass, df_fuel, left_on='op_unique_carrier',
right_on='unique_carrier', how='left').drop('unique_carrier', axis=1)

In [25]:
# drop missing fuel data
df_fpf.dropna(inplace=True)

### Merge with cleaned Weather Data

In [26]:
df_weather = pd.read_csv('data/processed/weather2020_cleaned.csv', index_col=[0])

In [27]:
df_fpf['weather_id']=df_fpf['fl_date'].astype(str) + "-" + df_fpf['origin']

In [28]:
df_flw = pd.merge(df_fpf, df_weather, left_on='weather_id', right_on='date_orig_id', how='left')

In [29]:
df_flw = df_flw.groupby(['origin_city','fl_date']).apply(lambda x: x.ffill().bfill())

In [30]:
df_flw.dropna(inplace=True)

In [31]:
df_flw=df_flw[['mkt_unique_carrier', 'origin','origin_city',
    'dest', 'dest_city', 'crs_elapsed_time','crs_dep_hour',
    'dep_tod', 'crs_arr_hour', 'arr_tod', 'distance', 'distance_group', 
    'fl_date', 'fl_day', 'week_num', 'departures_performed', 'payload', 
    'passengers', 'freight', 'sdomt_gallons', 'tdomt_gallons','sdomt_cost', 
    'tdomt_cost', 'AWND','PRCP', 'SNOW', 'SNWD', 'TAVG']]

In [32]:
df_flw.to_csv('test_flights_complete_raw_7day.csv')